In [ ]:
# Informatica / Data Stage /  SSIS / Python (ETL)

In [1]:
import pandas as pd
import sqlite3
import json
import pyodbc
from datetime import datetime

In [2]:
students_file = 'data/students.txt'     
grades_file = 'data/api_grades.json'     
sqlite_db = 'data/courses.db'          

In [ ]:
# 1. Extract 

In [3]:
students_df = pd.read_csv(students_file, sep='|')
print("Students Extracted:")
print(students_df.head())

Students Extracted:
   student_id          name gender        city
0           1   Mohamed Ali      M  Alexandria
1           2  Amira Hassan      F       Cairo
2           3   Karim Saeed      M       Aswan
3           4  Sara Mohamed      F  Alexandria
4           5     Omar Adel      M       Cairo


In [4]:
students_df

,student_id,name,gender,city
0,1,Mohamed Ali,M,Alexandria
1,2,Amira Hassan,F,Cairo
2,3,Karim Saeed,M,Aswan
3,4,Sara Mohamed,F,Alexandria
4,5,Omar Adel,M,Cairo
5,6,Nour Youssef,F,Aswan
6,7,Tarek Ibrahim,M,Alexandria
7,8,Laila Samir,F,Cairo
8,9,Hassan Farouk,M,Aswan
9,10,Mona Fathy,F,Alexandria


In [5]:
conn = sqlite3.connect(sqlite_db)
cursor = conn.cursor()
cursor.execute("SELECT * FROM courses")
courses_data = cursor.fetchall()
courses_df = pd.DataFrame(courses_data, columns=['course_id', 'course_name','credits'])
cursor.close()
conn.close()

print("Courses Extracted:")
print(courses_df.head())

Courses Extracted:
   course_id                  course_name  credits
0        201                Mathematics I        3
1        202                    Physics I        3
2        203  Introduction to Programming        4
3        204                    Databases        3
4        205         Software Engineering        3


In [6]:
courses_df

,course_id,course_name,credits
0,201,Mathematics I,3
1,202,Physics I,3
2,203,Introduction to Programming,4
3,204,Databases,3
4,205,Software Engineering,3


In [7]:
with open(grades_file, 'r') as f:
    grades_data = json.load(f)
grades_df = pd.DataFrame(grades_data)

print("Grades Extracted:")
print(grades_df.head())

Grades Extracted:
   student_id  course_id  grade  attendance        date
0           1        201     85          90  2024-01-15
1           2        202     78          85  2024-01-18
2           3        203     92          95  2024-02-02
3           4        204     88          92  2024-02-10
4           5        205     74          80  2024-03-05


In [8]:
grades_df

,student_id,course_id,grade,attendance,date
0,1,201,85,90,2024-01-15
1,2,202,78,85,2024-01-18
2,3,203,92,95,2024-02-02
3,4,204,88,92,2024-02-10
4,5,205,74,80,2024-03-05
5,6,201,81,88,2024-03-12
6,7,202,69,70,2024-04-01
7,8,203,95,98,2024-04-07
8,9,204,87,90,2024-05-03
9,10,205,90,93,2024-05-20


In [ ]:
# 2. Transform

In [14]:
grades_df['Date'] = pd.to_datetime(grades_df['date'])
grades_df['Date']

0   2024-01-15
1   2024-01-18
2   2024-02-02
3   2024-02-10
4   2024-03-05
5   2024-03-12
6   2024-04-01
7   2024-04-07
8   2024-05-03
9   2024-05-20
Name: Date, dtype: datetime64[ns]

In [ ]:
grades_df['Date'] = pd.to_datetime(grades_df['date'])
grades_df['Date']


0   2024-01-15
1   2024-01-18
2   2024-02-02
3   2024-02-10
4   2024-03-05
5   2024-03-12
6   2024-04-01
7   2024-04-07
8   2024-05-03
9   2024-05-20
Name: Date, dtype: datetime64[ns]

In [16]:
dim_date = grades_df[['Date']].drop_duplicates().reset_index(drop=True)
dim_date

,Date
0,2024-01-15
1,2024-01-18
2,2024-02-02
3,2024-02-10
4,2024-03-05
5,2024-03-12
6,2024-04-01
7,2024-04-07
8,2024-05-03
9,2024-05-20


In [ ]:
dim_date['DateKey'] = dim_date['Date'].dt.strftime('%Y%m%d').astype(int)
dim_date['DateKey'] 

0    20240115
1    20240118
2    20240202
3    20240210
4    20240305
5    20240312
6    20240401
7    20240407
8    20240503
9    20240520
Name: DateKey, dtype: int32

In [18]:
dim_date[['Year','Month','Day']] = dim_date['Date'].dt.strftime('%Y-%m-%d').str.split('-', expand=True).astype(int)


In [19]:
dim_date

,Date,DateKey,Year,Month,Day
0,2024-01-15,20240115,2024,1,15
1,2024-01-18,20240118,2024,1,18
2,2024-02-02,20240202,2024,2,2
3,2024-02-10,20240210,2024,2,10
4,2024-03-05,20240305,2024,3,5
5,2024-03-12,20240312,2024,3,12
6,2024-04-01,20240401,2024,4,1
7,2024-04-07,20240407,2024,4,7
8,2024-05-03,20240503,2024,5,3
9,2024-05-20,20240520,2024,5,20


,student_id,course_id,grade,attendance,date,Date
0,1,201,85,90,2024-01-15,2024-01-15
1,2,202,78,85,2024-01-18,2024-01-18
2,3,203,92,95,2024-02-02,2024-02-02
3,4,204,88,92,2024-02-10,2024-02-10
4,5,205,74,80,2024-03-05,2024-03-05
5,6,201,81,88,2024-03-12,2024-03-12
6,7,202,69,70,2024-04-01,2024-04-01
7,8,203,95,98,2024-04-07,2024-04-07
8,9,204,87,90,2024-05-03,2024-05-03
9,10,205,90,93,2024-05-20,2024-05-20


In [23]:
fact_df = grades_df.merge(students_df[['student_id']], on='student_id', how='left')
fact_df = fact_df.merge(courses_df[['course_id']], on='course_id', how='left')
fact_df = fact_df.merge(dim_date[['Date', 'DateKey']], on='Date', how='left')
fact_df = fact_df[['student_id', 'course_id', 'grade', 'attendance', 'DateKey']]

fact_df

,student_id,course_id,grade,attendance,DateKey
0,1,201,85,90,20240115
1,2,202,78,85,20240118
2,3,203,92,95,20240202
3,4,204,88,92,20240210
4,5,205,74,80,20240305
5,6,201,81,88,20240312
6,7,202,69,70,20240401
7,8,203,95,98,20240407
8,9,204,87,90,20240503
9,10,205,90,93,20240520


In [24]:
fact_df = fact_df.astype({
    'student_id': int,
    'course_id': int,
    'grade': int,
    'attendance': int,
    'DateKey': int
})

In [ ]:
# 3. Load

In [25]:
import pyodbc

server = 'localhost'
database = 'StudentPerformanceDW'
username = 'sa'
password = 'sa'
driver = '{ODBC Driver 17 for SQL Server}'

conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')

In [26]:
students_df

,student_id,name,gender,city
0,1,Mohamed Ali,M,Alexandria
1,2,Amira Hassan,F,Cairo
2,3,Karim Saeed,M,Aswan
3,4,Sara Mohamed,F,Alexandria
4,5,Omar Adel,M,Cairo
5,6,Nour Youssef,F,Aswan
6,7,Tarek Ibrahim,M,Alexandria
7,8,Laila Samir,F,Cairo
8,9,Hassan Farouk,M,Aswan
9,10,Mona Fathy,F,Alexandria


In [27]:
students_data = students_df[['student_id', 'name', 'gender', 'city']].astype(str).values.tolist()
students_data

[['1', 'Mohamed Ali', 'M', 'Alexandria'],
 ['2', 'Amira Hassan', 'F', 'Cairo'],
 ['3', 'Karim Saeed', 'M', 'Aswan'],
 ['4', 'Sara Mohamed', 'F', 'Alexandria'],
 ['5', 'Omar Adel', 'M', 'Cairo'],
 ['6', 'Nour Youssef', 'F', 'Aswan'],
 ['7', 'Tarek Ibrahim', 'M', 'Alexandria'],
 ['8', 'Laila Samir', 'F', 'Cairo'],
 ['9', 'Hassan Farouk', 'M', 'Aswan'],
 ['10', 'Mona Fathy', 'F', 'Alexandria']]

In [28]:
courses_data = courses_df[['course_id', 'course_name', 'credits']].astype(object).values.tolist()
dates_data = dim_date[['DateKey', 'Date', 'Year', 'Month', 'Day']].copy()
dates_data['Date'] = dates_data['Date'].dt.date
dates_data = dates_data.values.tolist()
fact_data = fact_df.values.tolist()


In [29]:
students_data

[['1', 'Mohamed Ali', 'M', 'Alexandria'],
 ['2', 'Amira Hassan', 'F', 'Cairo'],
 ['3', 'Karim Saeed', 'M', 'Aswan'],
 ['4', 'Sara Mohamed', 'F', 'Alexandria'],
 ['5', 'Omar Adel', 'M', 'Cairo'],
 ['6', 'Nour Youssef', 'F', 'Aswan'],
 ['7', 'Tarek Ibrahim', 'M', 'Alexandria'],
 ['8', 'Laila Samir', 'F', 'Cairo'],
 ['9', 'Hassan Farouk', 'M', 'Aswan'],
 ['10', 'Mona Fathy', 'F', 'Alexandria']]

In [ ]:
cursor = conn.cursor()


# Insert into dim_student (includes City)
cursor.executemany(
    "INSERT INTO dim_student (StudentID, Name, Gender, City) VALUES (?, ?, ?, ?)",
    students_data
)


In [32]:
conn.commit()

In [33]:


# Insert into dim_course
cursor.executemany(
    "INSERT INTO dim_course (CourseID, CourseName, Credits) VALUES (?, ?, ?)",
    courses_data
)

# Insert into dim_date
cursor.executemany(
    "INSERT INTO dim_date (DateKey, FullDate, Year, Month, Day) VALUES (?, ?, ?, ?, ?)",
    dates_data
)

# Insert into fact_student_performance
cursor.executemany(
    """INSERT INTO fact_student_performance (StudentID, CourseID, Grade, Attendance, DateKey)
       VALUES (?, ?, ?, ?, ?)""",
    fact_data
)

conn.commit()
cursor.close()
conn.close()

print("✅ ETL Completed Successfully!")

✅ ETL Completed Successfully!
